# Going Deep: Why Depth Matters

_Adapted from [Dataflowr Module 14a](https://dataflowr.github.io/website/modules/14a-depth/) by Andrei Bursuc._

**Why should we stack more layers?**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Use GPU if available
device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Using device: {device}")

## Datasets

Let's start with 2D datasets of increasing complexity for a binary classification task (red/blue points).

In [ ]:
def make_linear_data(n=500):
    """Two Gaussian clusters, linearly separable."""
    X0 = torch.randn(n // 2, 2) * 0.6 + torch.tensor([-1.0, -1.0])
    X1 = torch.randn(n // 2, 2) * 0.6 + torch.tensor([1.0, 1.0])
    X = torch.cat([X0, X1], dim=0)
    y = torch.cat([torch.zeros(n // 2), torch.ones(n // 2)]).long()
    return X, y

def make_circle_data(n=500):
    """Inner circle (class 0) vs outer ring (class 1)."""
    theta = torch.linspace(0, 2 * np.pi, n // 2)
    r_inner = torch.randn(n // 2) * 0.1 + 0.5
    r_outer = torch.randn(n // 2) * 0.1 + 1.5
    X_inner = torch.stack([r_inner * torch.cos(theta), r_inner * torch.sin(theta)], dim=1)
    X_outer = torch.stack([r_outer * torch.cos(theta), r_outer * torch.sin(theta)], dim=1)
    X = torch.cat([X_inner, X_outer], dim=0)
    y = torch.cat([torch.zeros(n // 2), torch.ones(n // 2)]).long()
    return X, y

def make_spiral_data(n=500):
    """Two interleaved spirals — a classic hard 2D problem."""
    t = torch.linspace(0, 4 * np.pi, n // 2)
    r = t / (4 * np.pi) * 2
    noise = torch.randn(n // 2) * 0.08
    X0 = torch.stack([(r + noise) * torch.cos(t), (r + noise) * torch.sin(t)], dim=1)
    X1 = torch.stack([(r + noise) * torch.cos(t + np.pi), (r + noise) * torch.sin(t + np.pi)], dim=1)
    X = torch.cat([X0, X1], dim=0)
    y = torch.cat([torch.zeros(n // 2), torch.ones(n // 2)]).long()
    return X, y

In [ ]:
# Visualize all three datasets
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
datasets = [
    ('Linear', make_linear_data),
    ('Circle', make_circle_data),
    ('Spiral', make_spiral_data)
]

for ax, (name, make_data) in zip(axes, datasets):
    X, y = make_data()
    ax.scatter(X[y == 0, 0], X[y == 0, 1], c='royalblue', alpha=0.5, s=10, label='Class 0')
    ax.scatter(X[y == 1, 0], X[y == 1, 1], c='crimson', alpha=0.5, s=10, label='Class 1')
    ax.set_title(name, fontsize=14)
    ax.set_aspect('equal')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

We use the standard PyTorch training loop with Adam optimizer (which we studied in [Chapter 6](../6-optimization/optimization.ipynb)). We train on the full batch (since our datasets are small) and track the loss and accuracy over epochs.

In [ ]:
def train_model(model, X, y, epochs=1000, lr=0.01):
    """
    Train a classification model using Adam and CrossEntropyLoss.
    Returns the training history (loss and accuracy per epoch).
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    history = {'loss': [], 'acc': []}
    
    for epoch in range(epochs):
        # Forward pass
        logits = model(X)
        loss = criterion(logits, y)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track metrics
        with torch.no_grad():
            preds = logits.argmax(dim=1)
            acc = (preds == y).float().mean().item()
            history['loss'].append(loss.item())
            history['acc'].append(acc)
    
    print(f"Final loss: {history['loss'][-1]:.4f}, accuracy: {history['acc'][-1]:.2%}")
    return history

In [ ]:
def plot_decision_boundary(model, X, y, ax=None, title=None, resolution=200):
    """
    Plot the decision boundary of a 2D classifier.
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))
    
    # Create grid
    margin = 0.5
    x_min, x_max = X[:, 0].min() - margin, X[:, 0].max() + margin
    y_min, y_max = X[:, 1].min() - margin, X[:, 1].max() + margin
    xx, yy = torch.meshgrid(
        torch.linspace(x_min, x_max, resolution),
        torch.linspace(y_min, y_max, resolution),
        indexing='xy'
    )
    grid = torch.stack([xx.flatten(), yy.flatten()], dim=1)
    
    # Predict on grid
    with torch.no_grad():
        logits = model(grid)
        preds = logits.argmax(dim=1).reshape(xx.shape)
    
    # Plot decision boundary
    cmap_bg = ListedColormap(['#AACCFF', '#FFAAAA'])
    ax.contourf(xx.numpy(), yy.numpy(), preds.numpy(), alpha=0.3, cmap=cmap_bg)
    
    # Plot data points
    ax.scatter(X[y == 0, 0], X[y == 0, 1], c='royalblue', alpha=0.5, s=10, edgecolors='k', linewidths=0.3)
    ax.scatter(X[y == 1, 0], X[y == 1, 1], c='crimson', alpha=0.5, s=10, edgecolors='k', linewidths=0.3)
    
    if title:
        ax.set_title(title, fontsize=12)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.2)
    return ax

## One Hidden Layer: The Universal Approximator

In 1989, George Cybenko proved a remarkable result:

> **Universal Approximation Theorem** (Cybenko, 1989; Hornik et al., 1989): A feedforward network with a single hidden layer containing a finite number of neurons can approximate any continuous function on compact subsets of $\mathbb{R}^n$ to arbitrary precision.

More precisely, for any continuous function $g: [0,1]^d \to \mathbb{R}$ and any $\epsilon > 0$, there exists a network:

$$
f(x) = \sum_{i=1}^{N} \alpha_i \, \sigma(w_i^T x + b_i)
$$

such that $\|f - g\|_\infty < \epsilon$, where $\sigma$ is any non-constant, bounded, monotonically-increasing continuous activation function.

This theorem says nothing about:
- How many neurons $N$ are needed (it may be astronomically large)
- Whether gradient descent can find the right parameters
- Whether the resulting network generalizes well

Let's verify experimentally that a single hidden layer can handle our non-linear problems.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (name, make_data) in zip(axes, datasets):
    X, y = make_data()
    model = nn.Sequential(
        nn.Linear(2, 100), nn.ReLU(), nn.Linear(100, 2)
    )  # 1 hidden layer with 100 neurons
    history = train_model(model, X, y, epochs=2000, lr=0.01)
    plot_decision_boundary(model, X, y, ax=ax, title=f'1 Hidden Layer (100) — {name}')

plt.tight_layout()
plt.show()

The Universal Approximation Theorem guarantees that a sufficiently wide single-layer network **exists** that can solve the spiral problem. But:
1. The required width may be very large
2. Optimization may fail to find the right parameters
3. The model may overfit with so many parameters

## Going Deeper: Depth vs Width

We can compare two strategies for allocating parameters:

- **Wide and shallow**: a single hidden layer with many neurons
- **Narrow and deep**: multiple hidden layers with fewer neurons each

In [ ]:
# Compare architectures with roughly similar parameter counts
architectures = [
    ('Wide-shallow  [2, 100, 2]', lambda: nn.Sequential(
        nn.Linear(2, 100), nn.ReLU(), nn.Linear(100, 2))),
    ('Medium        [2, 20, 20, 2]', lambda: nn.Sequential(
        nn.Linear(2, 20), nn.ReLU(), nn.Linear(20, 20), nn.ReLU(), nn.Linear(20, 2))),
    ('Deep-narrow   [2, 15, 15, 15, 2]', lambda: nn.Sequential(
        nn.Linear(2, 15), nn.ReLU(), nn.Linear(15, 15), nn.ReLU(),
        nn.Linear(15, 15), nn.ReLU(), nn.Linear(15, 2))),
]

print(f'{"Architecture":<45} {"Layers":>8} {"Params":>8}')
print('-' * 65)
for name, make_model in architectures:
    model = make_model()
    n_hidden = sum(1 for m in model if isinstance(m, nn.ReLU))
    n_params = sum(p.numel() for p in model.parameters())
    print(f'{name:<45} {n_hidden:>8} {n_params:>8}')

Now let us train all these architectures on the spiral dataset and compare their decision boundaries.

In [ ]:
X_spiral, y_spiral = make_spiral_data(n=1000)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
histories = []

for ax, (name, make_model) in zip(axes, architectures):
    torch.manual_seed(42)
    model = make_model()
    n_hidden = sum(1 for m in model if isinstance(m, nn.ReLU))
    n_params = sum(p.numel() for p in model.parameters())
    history = train_model(model, X_spiral, y_spiral, epochs=3000, lr=0.005)
    histories.append((n_hidden, n_params, history))
    plot_decision_boundary(model, X_spiral, y_spiral, ax=ax,
                          title=f'{n_hidden} hidden, {n_params} params')

plt.tight_layout()
plt.show()

Here are the training curves.

In [ ]:
# Training curves comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for n_hidden, n_params, history in histories:
    label = f'{n_hidden} hidden ({n_params} params)'
    ax1.plot(history['loss'], label=label, alpha=0.8)
    ax2.plot(history['acc'], label=label, alpha=0.8)

ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Hierarchical Features: What Each Layer Learns

Formally, a deep network computes a composition of functions:

$$
f(x) = f_L \circ f_{L-1} \circ \cdots \circ f_1(x)
$$

where each $f_\ell(z) = \sigma(W_\ell z + b_\ell)$ is a layer that:
1. Applies a linear transformation (rotation, scaling, shearing)
2. Applies a non-linear activation (ReLU folds the space)

Each layer transforms the data, making it progressively easier for the subsequent layers to classify.

In [ ]:
class MLPWithIntermediates(nn.Module):
    """An MLP that stores intermediate activations for visualization."""
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 8), nn.ReLU(),
            nn.Linear(8, 8), nn.ReLU(),
            nn.Linear(8, 8), nn.ReLU(),
            nn.Linear(8, 2),
        )

    def forward(self, x):
        self.intermediates = [x.detach().clone()]
        for layer in self.model:
            x = layer(x)
            if isinstance(layer, nn.ReLU):
                self.intermediates.append(x.detach().clone())
        self.intermediates.append(x.detach().clone())
        return x

    def get_intermediates(self, x):
        """Forward pass that returns all intermediate activations."""
        with torch.no_grad():
            self.forward(x)
        return self.intermediates

In [ ]:
# Train a deep network with 8D hidden layers for visualization
torch.manual_seed(0)
X_spiral, y_spiral = make_spiral_data(n=1000)

model_vis = MLPWithIntermediates()

# Train
optimizer = torch.optim.Adam(model_vis.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

for epoch in range(5000):
    logits = model_vis(X_spiral)
    loss = criterion(logits, y_spiral)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

with torch.no_grad():
    preds = model_vis(X_spiral).argmax(dim=1)
    acc = (preds == y_spiral).float().mean()
    print(f"Accuracy: {acc:.2%}")

In [ ]:
# Visualize intermediate representations (PCA projection to 2D)
from sklearn.decomposition import PCA

intermediates = model_vis.get_intermediates(X_spiral)

fig, axes = plt.subplots(1, len(intermediates), figsize=(4 * len(intermediates), 4))
layer_names = ['Input'] + [f'After Layer {i+1}' for i in range(len(intermediates) - 1)]

for i, (ax, Z, name) in enumerate(zip(axes, intermediates, layer_names)):
    Z_np = Z.numpy()
    if Z_np.shape[1] > 2:
        Z_2d = PCA(n_components=2).fit_transform(Z_np)
    else:
        Z_2d = Z_np
    ax.scatter(Z_2d[y_spiral == 0, 0], Z_2d[y_spiral == 0, 1],
              c='royalblue', alpha=0.5, s=10)
    ax.scatter(Z_2d[y_spiral == 1, 0], Z_2d[y_spiral == 1, 1],
              c='crimson', alpha=0.5, s=10)
    subtitle = f'{name}\n(PCA)' if Z_np.shape[1] > 2 else name
    ax.set_title(subtitle, fontsize=12)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

plt.suptitle('How each layer transforms the spiral data', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

_Note_: The hidden representations are 8-dimensional; we use PCA to project them down to 2D for visualization.

## Depth Efficiency: More with Less

### Intuition

Each ReLU layer can "fold" the space, and these folds compose multiplicatively.

- **1 fold** (1 layer): creates 2 regions
- **2 folds** (2 layers): creates 4 regions
- **$k$ folds** ($k$ layers): creates $2^k$ regions

A deep network with $k$ layers can carve the input space into $O(2^k)$ linear regions, while a shallow network with $N$ neurons can create at most $O(N)$ regions.

To illustrate this, let's consider a checkerboard dataset where the number of regions grows, and compare how depth and width handle it.

In [ ]:
def make_checkerboard_data(n=2000, freq=2):
    """
    Checkerboard pattern: class is determined by
    (floor(freq*x) + floor(freq*y)) % 2.
    Higher freq = more regions = harder problem.
    """
    X = torch.rand(n, 2) * 2 - 1  # Uniform in [-1, 1]^2
    y = ((torch.floor((X[:, 0] + 1) * freq) + torch.floor((X[:, 1] + 1) * freq)) % 2).long()
    return X, y

# Visualize checkerboards of increasing complexity
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, freq in zip(axes, [1, 2, 3]):
    X, y = make_checkerboard_data(freq=freq)
    ax.scatter(X[y == 0, 0], X[y == 0, 1], c='royalblue', alpha=0.3, s=5)
    ax.scatter(X[y == 1, 0], X[y == 1, 1], c='crimson', alpha=0.3, s=5)
    n_regions = (2 * freq) ** 2
    ax.set_title(f'Checkerboard freq={freq} ({n_regions} regions)', fontsize=12)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Now let us compare a wide-shallow model with a deep-narrow model on checkerboards of increasing complexity.

In [ ]:
# Depth efficiency experiment
freqs = [1, 2, 3]
configs = {
    'Shallow (1 hidden)': lambda: nn.Sequential(
        nn.Linear(2, 100), nn.ReLU(), nn.Linear(100, 2)),
    'Deep (4 hidden)': lambda: nn.Sequential(
        nn.Linear(2, 20), nn.ReLU(), nn.Linear(20, 20), nn.ReLU(),
        nn.Linear(20, 20), nn.ReLU(), nn.Linear(20, 20), nn.ReLU(),
        nn.Linear(20, 2)),
}

fig, axes = plt.subplots(len(configs), len(freqs), figsize=(5 * len(freqs), 5 * len(configs)))

for i, (config_name, make_model_fn) in enumerate(configs.items()):
    for j, freq in enumerate(freqs):
        torch.manual_seed(42)
        X, y = make_checkerboard_data(n=2000, freq=freq)
        model = make_model_fn()
        n_params = sum(p.numel() for p in model.parameters())
        history = train_model(model, X, y, epochs=2000, lr=0.005)
        ax = axes[i, j]
        plot_decision_boundary(model, X, y, ax=ax,
            title=f'{config_name}\nfreq={freq}, acc={history["acc"][-1]:.0%}, params={n_params}')

plt.tight_layout()
plt.show()